In [ ]:
 !pip uninstall tf-keras
 !pip install keras-tuner
 !pip install tensorflow==2.16.1

In [ ]:
import keras
import tensorflow as tf
print("Keras Current Version:", keras.__version__, "Tensorflow Current Version:", tf.__version__)

Keras Current Version: 3.8.0 Tensorflow Current Version: 2.16.1


In [ ]:
import numpy as np
import pandas as pd
from joblib import dump, load
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.initializers import RandomNormal, RandomUniform, GlorotUniform, GlorotNormal, HeNormal
from keras.optimizers.schedules import ExponentialDecay
from keras_tuner import RandomSearch, GridSearch, BayesianOptimization
from keras_tuner.engine.hyperparameters import HyperParameters

random.seed(46)
np.random.seed(46)
tf.random.set_seed(46)

# import os
import time


In [ ]:
def preprocess_data(filepath):
    data = pd.read_csv(filepath)
    scaler = StandardScaler()
    X = scaler.fit_transform(data.drop('Outcome', axis=1))
    y = data['Outcome'].values
    dump(scaler, 'scaler.joblib')
    return X, y

def prepare_datasets(X_train, X_val, y_train, y_val, batch_size=None):
    if batch_size is None:
        batch_size = len(X_train)
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(batch_size)
    val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
    val_dataset = val_dataset.batch(batch_size)
    return train_dataset, val_dataset

def plot_training_history(history, train_loss='loss', train_metric='accuracy', val_loss='val_loss', val_metric='val_accuracy'):

    #Loss
    plt.figure(figsize=(10, 5))
    plt.plot(history.history[train_loss], label='Training Loss')
    plt.plot(history.history[val_loss], label='Validation Loss')
    plt.title('Training and Validation Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    # Metrics
    plt.figure(figsize=(10, 5))
    plt.plot(history.history[train_metric], label=f"Training: {train_metric}")
    plt.plot(history.history[val_metric], label=f"Validation: {val_metric}")
    plt.title(f'Training and Validation {train_metric} Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel(f'train_metric')
    plt.legend()
    plt.show()

def get_best_epoch_details(history):
    val_losses = history.history['val_loss']
    min_val_loss_index = val_losses.index(min(val_losses))
    best_epoch = min_val_loss_index + 1

    epoch_details = {}
    for key in history.history.keys():
        epoch_details[key] = history.history[key][min_val_loss_index]

    epoch_details['best_epoch'] = best_epoch
    print(f"Best epoch details: {epoch_details}")

In [ ]:
X, y = preprocess_data('/content/drive/MyDrive/diabetes.csv adlı dosyanın kopyası')

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

train_ds, val_ds = prepare_datasets(X_train, X_val, y_train, y_val, batch_size=32)

In [ ]:
def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(train_ds.element_spec[0].shape[1],)))

    # Hidden layers, activation functions, l2, Dropout
    for i in range(hp.Int('num_layers', 1, 10)):

        model.add(Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=16),
                        activation=hp.Choice('activation_' + str(i), values=['relu', 'tanh', 'sigmoid']),
                        kernel_regularizer=l2(hp.Float('l2_' + str(i), min_value=0.0001, max_value=0.01, sampling='log'))))

        model.add(BatchNormalization())
        model.add(Dropout(hp.Float('dropout_' + str(i), min_value=0.0, max_value=0.5, step=0.05)))

    model.add(Dense(1, activation='sigmoid'))

    # Learning rate schedule
    initial_learning_rate = hp.Float('initial_learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')

    lr_schedule = ExponentialDecay(
        initial_learning_rate=initial_learning_rate,
        decay_steps=20,
        decay_rate=0.96,
        staircase=True
    )

    # optimizers
    optimizer_choice = hp.Choice('optimizer', values=['sgd', 'adam', "rmsprop"])
    if optimizer_choice == 'sgd':
        optimizer = SGD(
            learning_rate=lr_schedule,
            momentum=hp.Float('momentum', min_value=0.0, max_value=0.9, step=0.1)
        )
    elif optimizer_choice == 'adam':
        optimizer = Adam(
            learning_rate=lr_schedule,
            beta_1=hp.Float('beta1', min_value=0.85, max_value=0.99, step=0.01),
            beta_2=hp.Float('beta2', min_value=0.999, max_value=0.9999, step=0.0001),
            epsilon=hp.Float('epsilon', min_value=1e-8, max_value=1e-7, step=1e-8)
        )

    elif optimizer_choice == 'rmsprop':
        optimizer = RMSprop(
            learning_rate=lr_schedule,
            rho=hp.Float('rho', min_value=0.8, max_value=0.99, step=0.01),
            epsilon=hp.Float('epsilon', min_value=1e-10, max_value=1e-8, step=1e-10),
            momentum=hp.Float('momentum', min_value=0.0, max_value=0.9, step=0.1)
        )

    model.compile(optimizer=optimizer,
                  loss="binary_crossentropy",
                  metrics=["accuracy"])

    return model

In [ ]:
random_search_tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=20,
    executions_per_trial=1,
    overwrite=True)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=20,
    verbose=1,
    restore_best_weights=True)

In [ ]:
random_search_tuner.search(train_ds,
                           epochs=200,
                           validation_data=val_ds,
                           callbacks=[early_stopping])


Trial 5 Complete [00h 00m 56s]
val_loss: 4.7252607345581055

Best val_loss So Far: 0.5330563187599182
Total elapsed time: 00h 02m 58s

Search: Running Trial #6

Value             |Best Value So Far |Hyperparameter
9                 |5                 |num_layers
256               |240               |units_0
sigmoid           |sigmoid           |activation_0
0.005041          |0.0004401         |l2_0
0                 |0.15              |dropout_0
0.00042274        |0.0047134         |initial_learning_rate
rmsprop           |rmsprop           |optimizer
0.5               |0.3               |momentum
480               |224               |units_1
relu              |relu              |activation_1
0.00026657        |0.0037228         |l2_1
0.4               |0.25              |dropout_1
48                |272               |units_2
relu              |tanh              |activation_2
0.0097684         |0.0072299         |l2_2
0.35              |0.05              |dropout_2
288               

KeyboardInterrupt: 

In [ ]:
random_search_tuner.search_space_summary()

Search space summary
Default search space size: 40
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 10, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 16, 'sampling': 'linear'}
activation_0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}
l2_0 (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}
dropout_0 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.05, 'sampling': 'linear'}
initial_learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}
optimizer (Choice)
{'default': 'sgd', 'conditions': [], 'values': ['sgd', 'adam', 'rmsprop'], 'ordered': False}
momentum (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.9, 'ste

In [ ]:
best_hps = random_search_tuner.get_best_hyperparameters(num_trials=3)

In [ ]:
best_hps_1 = best_hps[0]
best_hps_2 = best_hps[1]
best_hps_3 = best_hps[2]

In [ ]:
print(f"Best hyperparameters: {best_hps_1.values}")
print(f"Best hyperparameters: {best_hps_2.values}")
print(f"Best hyperparameters: {best_hps_3.values}")

Best hyperparameters: {'num_layers': 5, 'units_0': 240, 'activation_0': 'sigmoid', 'l2_0': 0.00044010071253490897, 'dropout_0': 0.15000000000000002, 'initial_learning_rate': 0.004713373094066093, 'optimizer': 'rmsprop', 'momentum': 0.30000000000000004, 'units_1': 224, 'activation_1': 'relu', 'l2_1': 0.003722759302364584, 'dropout_1': 0.25, 'units_2': 272, 'activation_2': 'tanh', 'l2_2': 0.0072299017874572335, 'dropout_2': 0.05, 'units_3': 304, 'activation_3': 'sigmoid', 'l2_3': 0.0006120105803329905, 'dropout_3': 0.15000000000000002, 'units_4': 128, 'activation_4': 'tanh', 'l2_4': 0.005174671499174863, 'dropout_4': 0.2, 'units_5': 96, 'activation_5': 'tanh', 'l2_5': 0.0006628340069463782, 'dropout_5': 0.45, 'units_6': 256, 'activation_6': 'sigmoid', 'l2_6': 0.0009597364113807246, 'dropout_6': 0.1, 'units_7': 96, 'activation_7': 'tanh', 'l2_7': 0.0004988198425088675, 'dropout_7': 0.25, 'rho': 0.8, 'epsilon': 4.9e-09}
Best hyperparameters: {'num_layers': 8, 'units_0': 384, 'activation_0'

In [ ]:
best_models = random_search_tuner.get_best_models(num_models=3)

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 1 variables whereas the saved optimizer has 45 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'SGD', because it has 1 variables whereas the saved optimizer has 35 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 1 variables whereas the saved optimizer has 7 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
for i, model in enumerate(best_models):
    loss, acc = model.evaluate(val_ds, verbose=0)
    print(f"Model {i+1}, Validation loss: {loss}, Validation Accuracy: {acc}")

Model 1, Validation loss: 0.5330563187599182, Validation Accuracy: 0.7597402334213257
Model 2, Validation loss: 0.5978400707244873, Validation Accuracy: 0.7337662577629089
Model 3, Validation loss: 0.6432857513427734, Validation Accuracy: 0.7402597665786743


In [ ]:
# Model 1, Validation loss: 0.505858302116394, Validation Accuracy: 0.7792207598686218
# Model 2, Validation loss: 0.520683228969574, Validation Accuracy: 0.7792207598686218
# Model 3, Validation loss: 0.5239221453666687, Validation Accuracy: 0.7922077775001526